In this notebook we inspect the moonboard database and clean the data.  First we import some libraries and load the dataframe.

## Importing and cleaning the data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import re

# Set some default plotting params
%matplotlib inline
sns.set(style="white")
plt.style.use('seaborn-white')
plt.rcParams["image.origin"] = 'lower'
plt.rcParams['figure.figsize'] = (10.0, 8.0)

path_pickle = 'data/database.zip'
df = pd.read_pickle(path_pickle)

Let's first inspect the data and see what we have here.

In [2]:
df.head()

,ApiId,Attempts,DateDeleted,DateInserted,DateTimeString,DateUpdated,FirstAscender,Grade,Holdsets,Holdsetup,...,Name,NameForUrl,NumberOfTries,ProblemType,Rating,RepeatText,Repeats,Setter,UserGrade,UserRating
0,0,0,None,/Date(1542102209893)/,13 Nov 2018 09:43,None,False,6C,None,"{'Id': 15, 'Description': 'MoonBoard Masters 2...",...,WOODEN CENTERPEICE,wooden-centerpeice,None,None,0,Be the first to repeat this problem,0,"{'Id': '9384ECB7-94E4-4B83-AF0D-EE91EE11AB55',...",None,0
1,0,0,None,/Date(1542101683707)/,13 Nov 2018 09:34,None,False,7A+,None,"{'Id': 15, 'Description': 'MoonBoard Masters 2...",...,MUSE IN KOALA,muse-in-koala,None,None,0,1 climber has repeated this problem,1,"{'Id': '9384ECB7-94E4-4B83-AF0D-EE91EE11AB55',...",None,3
2,0,0,None,/Date(1542101635793)/,13 Nov 2018 09:33,None,False,6B+,None,"{'Id': 15, 'Description': 'MoonBoard Masters 2...",...,ORIGINAL TASTE,original-taste,None,None,0,1 climber has repeated this problem,1,"{'Id': '9384ECB7-94E4-4B83-AF0D-EE91EE11AB55',...",None,3
3,0,0,None,/Date(1542094956373)/,13 Nov 2018 07:42,None,False,6A+,None,"{'Id': 15, 'Description': 'MoonBoard Masters 2...",...,SOFT SERVE,soft-serve,None,None,0,Be the first to repeat this problem,0,"{'Id': '33a82150-f883-4f46-a3a7-da776da718dd',...",None,0
4,0,0,None,/Date(1542077121190)/,13 Nov 2018 02:45,None,False,7B+,None,"{'Id': 15, 'Description': 'MoonBoard Masters 2...",...,DOBRA,dobra,None,None,0,Be the first to repeat this problem,0,"{'Id': 'B0794F91-269D-4C5A-8113-366D005A97DD',...",None,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13570 entries, 0 to 13569
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ApiId                     13570 non-null  int64 
 1   Attempts                  13570 non-null  int64 
 2   DateDeleted               0 non-null      object
 3   DateInserted              13570 non-null  object
 4   DateTimeString            13570 non-null  object
 5   DateUpdated               0 non-null      object
 6   FirstAscender             13570 non-null  bool  
 7   Grade                     13570 non-null  object
 8   Holdsets                  0 non-null      object
 9   Holdsetup                 13570 non-null  object
 10  Id                        13570 non-null  int64 
 11  IsAssessmentProblem       13570 non-null  bool  
 12  IsBenchmark               13570 non-null  bool  
 13  Locations                 13570 non-null  object
 14  Method                

Let's drop the columns that we aren't interested in.  What we are interested in are the holds/how the moon board is configured, the grade (from the setter and users) and any other interesting bits of data that might be useful later on - such as if it's been repeated or if you are allowed to use holds for your feet or not.  Some columns such as `Holdsets` contain no data.

In [4]:
# create a list of the columns to keep
keep_list = ['DateTimeString', 'Grade', 'IsBenchmark', 'MoonBoardConfiguration', 'Name',
             'Setter', 'Method', 'Holdsetup', 'Repeats']
problem = df[keep_list]

# try to infer some better dtypes for the columns
problem.infer_objects()

problem.info()
problem.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13570 entries, 0 to 13569
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   DateTimeString          13570 non-null  object
 1   Grade                   13570 non-null  object
 2   IsBenchmark             13570 non-null  bool  
 3   MoonBoardConfiguration  13570 non-null  object
 4   Name                    13570 non-null  object
 5   Setter                  13570 non-null  object
 6   Method                  13570 non-null  object
 7   Holdsetup               13570 non-null  object
 8   Repeats                 13570 non-null  int64 
dtypes: bool(1), int64(1), object(7)
memory usage: 861.5+ KB


,DateTimeString,Grade,IsBenchmark,MoonBoardConfiguration,Name,Setter,Method,Holdsetup,Repeats
0,13 Nov 2018 09:43,6C,False,"{'Id': 1, 'Description': '40° MoonBoard', 'Low...",WOODEN CENTERPEICE,"{'Id': '9384ECB7-94E4-4B83-AF0D-EE91EE11AB55',...",Feet follow hands,"{'Id': 15, 'Description': 'MoonBoard Masters 2...",0
1,13 Nov 2018 09:34,7A+,True,"{'Id': 1, 'Description': '40° MoonBoard', 'Low...",MUSE IN KOALA,"{'Id': '9384ECB7-94E4-4B83-AF0D-EE91EE11AB55',...",Feet follow hands,"{'Id': 15, 'Description': 'MoonBoard Masters 2...",1
2,13 Nov 2018 09:33,6B+,False,"{'Id': 1, 'Description': '40° MoonBoard', 'Low...",ORIGINAL TASTE,"{'Id': '9384ECB7-94E4-4B83-AF0D-EE91EE11AB55',...",Feet follow hands,"{'Id': 15, 'Description': 'MoonBoard Masters 2...",1
3,13 Nov 2018 07:42,6A+,False,"{'Id': 1, 'Description': '40° MoonBoard', 'Low...",SOFT SERVE,"{'Id': '33a82150-f883-4f46-a3a7-da776da718dd',...",Feet follow hands + screw ons,"{'Id': 15, 'Description': 'MoonBoard Masters 2...",0
4,13 Nov 2018 02:45,7B+,False,"{'Id': 1, 'Description': '40° MoonBoard', 'Low...",DOBRA,"{'Id': 'B0794F91-269D-4C5A-8113-366D005A97DD',...",Footless + kickboard,"{'Id': 15, 'Description': 'MoonBoard Masters 2...",0


There are a couple of different configurations to the moon board but we will only keep the most popular one (where it's at a 40° overhang).  There are are a couple of different possibilities for footholds but again we'll only keep the most popular.

In [5]:
print(problem.iloc[0]['MoonBoardConfiguration'])
print(problem.iloc[0]['Setter'])

{'Id': 1, 'Description': '40° MoonBoard', 'LowGrade': None, 'HighGrade': None}
{'Id': '9384ECB7-94E4-4B83-AF0D-EE91EE11AB55', 'Nickname': 'KoalaClimbing', 'Firstname': 'ChaeYeon', 'Lastname': 'Lim', 'City': 'Seoul', 'Country': 'South Korea', 'ProfileImageUrl': '/Content/Account/Images/default-profile.png?636777030804646100', 'CanShareData': False}


In [6]:
problem['Setter'] = problem['Setter'].apply(lambda d: d['Id'])
problem['Holdsetup'] = problem['Holdsetup'].apply(lambda d: d['Description'])
problem['MoonBoardConfiguration'] = problem['MoonBoardConfiguration'].apply(lambda d: d['Description'])

<ipython-input-6-fbdacf9c90e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  problem['Setter'] = problem['Setter'].apply(lambda d: d['Id'])
<ipython-input-6-fbdacf9c90e0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  problem['Holdsetup'] = problem['Holdsetup'].apply(lambda d: d['Description'])
<ipython-input-6-fbdacf9c90e0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [7]:
keep_list = ['Holdsetup', 'Method', 'MoonBoardConfiguration']
moonboard_config = problem[keep_list]
moonboard_config.drop_duplicates(inplace=True)
moonboard_config.reset_index(drop=True, inplace=True)
    
print(moonboard_config)
moonboard_config.info()

                Holdsetup                         Method  \
0  MoonBoard Masters 2017              Feet follow hands   
1  MoonBoard Masters 2017  Feet follow hands + screw ons   
2  MoonBoard Masters 2017           Footless + kickboard   
3  MoonBoard Masters 2017              Feet follow hands   
4  MoonBoard Masters 2017  Feet follow hands + screw ons   
5  MoonBoard Masters 2017                 Screw ons only   
6  MoonBoard Masters 2017                 Screw ons only   
7  MoonBoard Masters 2017           Footless + kickboard   

  MoonBoardConfiguration  
0          40° MoonBoard  
1          40° MoonBoard  
2          40° MoonBoard  
3          25° MoonBoard  
4          25° MoonBoard  
5          40° MoonBoard  
6          25° MoonBoard  
7          25° MoonBoard  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Holdset

<ipython-input-7-500590046bbb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moonboard_config.drop_duplicates(inplace=True)


In [8]:
r,c = problem.shape
for i in range(r):
    problem.at[i, 'MoonBoardConfiguration'] = (moonboard_config[(moonboard_config['Method'] == problem.iloc[i]['Method']) & (moonboard_config['Holdsetup'] == problem.iloc[i]['Holdsetup']) & (moonboard_config['MoonBoardConfiguration'] == problem.iloc[i]['MoonBoardConfiguration'])].index.tolist()[0])

problem = problem.drop(columns=['Holdsetup', 'Method'])
problem.head()

,DateTimeString,Grade,IsBenchmark,MoonBoardConfiguration,Name,Setter,Repeats
0,13 Nov 2018 09:43,6C,False,0,WOODEN CENTERPEICE,9384ECB7-94E4-4B83-AF0D-EE91EE11AB55,0
1,13 Nov 2018 09:34,7A+,True,0,MUSE IN KOALA,9384ECB7-94E4-4B83-AF0D-EE91EE11AB55,1
2,13 Nov 2018 09:33,6B+,False,0,ORIGINAL TASTE,9384ECB7-94E4-4B83-AF0D-EE91EE11AB55,1
3,13 Nov 2018 07:42,6A+,False,1,SOFT SERVE,33a82150-f883-4f46-a3a7-da776da718dd,0
4,13 Nov 2018 02:45,7B+,False,2,DOBRA,B0794F91-269D-4C5A-8113-366D005A97DD,0


In [9]:
# PROBLEM and MOONBOARD_CONFIG are done. Now create SETTER table
temp = df['Setter'].to_dict()
setter = pd.DataFrame.from_dict(temp, orient='index')
setter.reset_index(drop=True, inplace=True)
del setter['ProfileImageUrl']
setter.drop_duplicates(subset=None, keep="first", inplace=True)
print(setter)

                                         Id       Nickname Firstname  \
0      9384ECB7-94E4-4B83-AF0D-EE91EE11AB55  KoalaClimbing  ChaeYeon   
3      33a82150-f883-4f46-a3a7-da776da718dd          Jtime    Justin   
4      B0794F91-269D-4C5A-8113-366D005A97DD          B. T.        B.   
5      6bb6f7a7-1c52-4eb4-ba78-b9446f5533ab         Cassie    Cassie   
6      015cc8b7-4676-48e9-abfb-3ffd8765c00c          jason     jason   
...                                     ...            ...       ...   
13495  05F55596-9321-4046-83BA-7D163EA9E919    Kyra Condie      Kyra   
13496  14e799d3-0671-4a7f-987d-53132bed3c8b          Katty     Katja   
13497  24845b44-dac4-44fb-b916-fa14d41598e1       Gelmanov    Rustam   
13499  9d270597-710a-4e26-9626-f1f88ac1ef16      margojain     Margo   
13535  77DA26C4-5B20-4D91-A357-ADB361F65F1A  jamie parbery     jamie   

       Lastname           City         Country  CanShareData  
0           Lim          Seoul     South Korea         False  
3        

In [10]:
#create MOVES table
dictionary = {}
j = 0
for i in range(r):
    temp = df['Moves'][i]
    
    # the loop
    for entry in temp:
        # add a dictionary entry to the final dictionary
        dictionary[j] = {"pId": i, "Description": entry['Description'], "IsStart": entry['IsStart'], "IsEnd": entry['IsEnd']}
    
        # increment the counter
        j = j + 1

moves = pd.DataFrame.from_dict(dictionary, "index")
moves.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109049 entries, 0 to 109048
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   pId          109049 non-null  int64 
 1   Description  109049 non-null  object
 2   IsStart      109049 non-null  bool  
 3   IsEnd        109049 non-null  bool  
dtypes: bool(2), int64(1), object(1)
memory usage: 2.7+ MB


In [11]:
moves.head()

,pId,Description,IsStart,IsEnd
0,0,C5,True,False
1,0,F6,True,False
2,0,D9,False,False
3,0,G12,False,False
4,0,C14,False,False


In [12]:
from sqlalchemy import create_engine

# Credentials to database connection
hostname="dbase.cs.jhu.edu"
dbname="20fa_egan1_db"
uname="20fa_egan1"
pwd="Xrck22yf3Z"

# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}?charset=utf8mb4".format(host=hostname, db=dbname, user=uname, pw=pwd))

# Add indices as columns in each dataframe
problem['pId'] = problem.index
moonboard_config['cId'] = moonboard_config.index
moves['mId'] = moves.index
setter['sId'] = setter.index

# Convert all dataframes to sql tables
problem.to_sql('Problem', engine, index=False)
moonboard_config.to_sql('MoonboardConfig', engine, index=False)
moves.to_sql('Moves', engine, index=False)
setter.to_sql('Setter', engine, index=False)